In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from textblob.sentiments import PatternAnalyzer
import regex as re

In [2]:
BH_tweets_during_pandemic = pd.read_csv('dane/BH_tweets_during_pandemic.csv', lineterminator='\n')

In [3]:
BH_tweets_during_pandemic.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video,near,geo,source,user_rt_id,user_rt,retweet_id,retweet_date,translate,trans_src,trans_dest
count,4.042400e+04,4.042400e+04,40424.0,4.042400e+04,40424.00000,40424.000000,40424.000000,40424.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1.276395e+18,1.276152e+18,200.0,3.864831e+17,0.40382,1.250940,4.858104,0.335271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1.912573e+16,2.135468e+16,0.0,5.085653e+17,3.21997,15.404715,56.520091,0.472091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.245139e+18,3.964446e+17,200.0,3.240000e+02,0.00000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.257207e+18,1.257182e+18,200.0,1.765739e+08,0.00000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.276260e+18,1.276206e+18,200.0,1.651895e+09,0.00000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.294312e+18,1.294248e+18,200.0,9.352342e+17,0.00000,0.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.311455e+18,1.311455e+18,200.0,1.310660e+18,189.00000,1049.000000,4178.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
BH_tweets_during_pandemic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40424 entries, 0 to 40423
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               40424 non-null  int64  
 1   conversation_id  40424 non-null  int64  
 2   created_at       40424 non-null  object 
 3   date             40424 non-null  object 
 4   time             40424 non-null  object 
 5   timezone         40424 non-null  int64  
 6   user_id          40424 non-null  int64  
 7   username         40424 non-null  object 
 8   name             40422 non-null  object 
 9   place            260 non-null    object 
 10  tweet            40424 non-null  object 
 11  language         40424 non-null  object 
 12  mentions         40424 non-null  object 
 13  urls             40424 non-null  object 
 14  photos           40424 non-null  object 
 15  replies_count    40424 non-null  int64  
 16  retweets_count   40424 non-null  int64  
 17  likes_count 

In [5]:
BH_tweets_during_pandemic.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1311454882008891393,1311454882008891393,2020-10-01 01:56:24 CEST,2020-10-01,01:56:24,200,1262819762662301696,sells_san,SueSellsSanDiego,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1311451801011531776,1311451801011531776,2020-10-01 01:44:09 CEST,2020-10-01,01:44:09,200,485907955,wmwpres,Dave (WildMan) Weber,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1311451427559157762,1311451427559157762,2020-10-01 01:42:40 CEST,2020-10-01,01:42:40,200,332364947,simonsalhanyre,BHHS Simon & Salhany,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1311450245226692609,1311450245226692609,2020-10-01 01:37:58 CEST,2020-10-01,01:37:58,200,221858096,yourchgorealtor,Carla Walker,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1311448341998383105,1311448341998383105,2020-10-01 01:30:24 CEST,2020-10-01,01:30:24,200,289964571,bhhspenfedks,BHHS PenFed Realty Kansas,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [6]:
#Add date format and sorted by dates
BH_tweets_during_pandemic['date'] = pd.to_datetime(BH_tweets_during_pandemic['date'], format = '%Y-%m-%d')
BH_tweets_during_pandemic = BH_tweets_during_pandemic.sort_values(by='date')
BH_tweets_during_pandemic = BH_tweets_during_pandemic.reset_index(drop=True)
BH_tweets_during_pandemic

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1245138896004042754,1245138896004042754,2020-04-01 02:00:21 CEST,2020-04-01,02:00:21,200,984538659234828289,paulfrinkbhhs,Berkshire Hathaway HomeServices NW - Federal Way,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1245423530461859840,1245423530461859840,2020-04-01 20:51:23 CEST,2020-04-01,20:51:23,200,750682473369972736,webuffett1,Warren & Charlie,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1245419168289210368,1245419168289210368,2020-04-01 20:34:03 CEST,2020-04-01,20:34:03,200,3248568151,wilkinson991,Daniel Wilkinson,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1245417140397707267,1245417140397707267,2020-04-01 20:25:59 CEST,2020-04-01,20:25:59,200,373883201,vladobotsvadze,Vladimer Botsvadze,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1245416924848238592,1245416924848238592,2020-04-01 20:25:08 CEST,2020-04-01,20:25:08,200,1666180093,themillerteam_,Michelle Miller,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40419,1311428662693163008,1311428662693163008,2020-10-01 00:12:13 CEST,2020-10-01,00:12:13,200,351881121,christophercud1,Chris,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
40420,1311426852825763840,1311426852825763840,2020-10-01 00:05:01 CEST,2020-10-01,00:05:01,200,351881121,christophercud1,Chris,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
40421,1311425775929831427,1311425775929831427,2020-10-01 00:00:44 CEST,2020-10-01,00:00:44,200,800033690474659840,financedpremium,Financed Premiums,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
40422,1311438444799893504,1311438444799893504,2020-10-01 00:51:05 CEST,2020-10-01,00:51:05,200,723604032141561856,dalesells30a,Dale Stackable,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [7]:
#Remove tweets writeen in other language than english
BH_tweets_during_pandemic_clean = BH_tweets_during_pandemic[BH_tweets_during_pandemic.language == 'en']
BH_tweets_during_pandemic_clean = BH_tweets_during_pandemic_clean.reset_index(drop=True)
BH_tweets_during_pandemic_clean.shape

(36155, 36)

In [8]:
#Remove columns without any data
BH_tweets_during_pandemic_clean = BH_tweets_during_pandemic_clean.dropna(thresh = 1, axis = 1)

In [9]:
#4984 records and 10 columns were removed
BH_tweets_during_pandemic_clean.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video
count,3.615500e+04,3.615500e+04,36155.0,3.615500e+04,36155.000000,36155.000000,36155.000000,36155.000000
mean,1.276440e+18,1.276178e+18,200.0,3.836063e+17,0.386696,1.200664,4.509335,0.351348
std,1.915956e+16,2.162785e+16,0.0,5.057701e+17,3.324907,15.906309,57.913247,0.477398
min,1.245139e+18,3.964446e+17,200.0,3.240000e+02,0.000000,0.000000,0.000000,0.000000
25%,1.257238e+18,1.257204e+18,200.0,1.805376e+08,0.000000,0.000000,0.000000,0.000000
50%,1.276289e+18,1.276226e+18,200.0,1.653948e+09,0.000000,0.000000,0.000000,0.000000
75%,1.294033e+18,1.293992e+18,200.0,9.213088e+17,0.000000,0.000000,1.000000,1.000000
max,1.311455e+18,1.311455e+18,200.0,1.310537e+18,189.000000,1049.000000,4178.000000,1.000000


In [10]:
BH_tweets_during_pandemic_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36155 entries, 0 to 36154
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               36155 non-null  int64         
 1   conversation_id  36155 non-null  int64         
 2   created_at       36155 non-null  object        
 3   date             36155 non-null  datetime64[ns]
 4   time             36155 non-null  object        
 5   timezone         36155 non-null  int64         
 6   user_id          36155 non-null  int64         
 7   username         36155 non-null  object        
 8   name             36153 non-null  object        
 9   place            253 non-null    object        
 10  tweet            36155 non-null  object        
 11  language         36155 non-null  object        
 12  mentions         36155 non-null  object        
 13  urls             36155 non-null  object        
 14  photos           36155 non-null  objec

In [11]:
BH_tweets_during_pandemic_clean.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to
0,1245138896004042754,1245138896004042754,2020-04-01 02:00:21 CEST,2020-04-01,02:00:21,200,984538659234828289,paulfrinkbhhs,Berkshire Hathaway HomeServices NW - Federal Way,NaN,...,0,0,"['wesellnwdreams', 'bhhsnwrefw', 'bhhsnwre', '...",[],https://twitter.com/paulfrinkbhhs/status/12451...,False,NaN,1,https://pbs.twimg.com/ext_tw_video_thumb/12451...,[]
1,1245423530461859840,1245423530461859840,2020-04-01 20:51:23 CEST,2020-04-01,20:51:23,200,750682473369972736,webuffett1,Warren & Charlie,NaN,...,0,0,[],[],https://twitter.com/WEBuffett1/status/12454235...,False,NaN,0,NaN,[]
2,1245419168289210368,1245419168289210368,2020-04-01 20:34:03 CEST,2020-04-01,20:34:03,200,3248568151,wilkinson991,Daniel Wilkinson,NaN,...,0,0,['investing'],[],https://twitter.com/wilkinson991/status/124541...,False,NaN,0,NaN,[]
3,1245417140397707267,1245417140397707267,2020-04-01 20:25:59 CEST,2020-04-01,20:25:59,200,373883201,vladobotsvadze,Vladimer Botsvadze,NaN,...,0,0,[],[],https://twitter.com/VladoBotsvadze/status/1245...,False,NaN,1,https://pbs.twimg.com/media/EUicSM1WkAUkg_v.jpg,[]
4,1245416924848238592,1245416924848238592,2020-04-01 20:25:08 CEST,2020-04-01,20:25:08,200,1666180093,themillerteam_,Michelle Miller,NaN,...,0,0,[],[],https://twitter.com/TheMillerTeam_/status/1245...,False,NaN,1,https://pbs.twimg.com/media/EUicF7nX0AMzwba.jpg,[]


In [12]:
#Check values of column place
BH_tweets_during_pandemic_clean['place'].value_counts()

{'type': 'Point', 'coordinates': [44.6981658, -93.2177749]}      60
{'type': 'Point', 'coordinates': [30.32487, -81.39882]}          14
{'type': 'Point', 'coordinates': [41.25877517, -96.19294167]}    10
{'type': 'Point', 'coordinates': [24.55693, -81.80389]}           4
{'type': 'Point', 'coordinates': [41.90067996, -87.62439966]}     4
                                                                 ..
{'type': 'Point', 'coordinates': [28.4812, -82.5436]}             1
{'type': 'Point', 'coordinates': [43.6006, -79.5053]}             1
{'type': 'Point', 'coordinates': [41.690255, -72.654812]}         1
{'type': 'Point', 'coordinates': [36.509, -4.8855]}               1
{'type': 'Point', 'coordinates': [43.5407948, -84.3331424]}       1
Name: place, Length: 131, dtype: int64

In [13]:
#Check values of column quote_url
BH_tweets_during_pandemic_clean['quote_url'].value_counts()

0                                                                 34
https://twitter.com/trengriffin/status/1256687728743202819         9
https://twitter.com/CNBCnow/status/1246193229546033152             6
https://twitter.com/YahooNews/status/1253419843954438146           5
https://twitter.com/CNBCnow/status/1256719898505601024             5
                                                                  ..
https://twitter.com/nytclimate/status/1279899968258617346          1
https://twitter.com/FraserMyers/status/1295375823361449985         1
https://twitter.com/MrGoodlyCooks/status/1290724306356449280       1
https://twitter.com/1thebusinesskid/status/1277240661628997634     1
https://twitter.com/ThisOldHouse/status/1256923714941325312        1
Name: quote_url, Length: 774, dtype: int64

In [14]:
#Counted quantity of links for each tweet
BH_tweets_during_pandemic_clean['quote_url'] = BH_tweets_during_pandemic_clean['quote_url'].str.count('https://')

#Replayce NaN by 0
BH_tweets_during_pandemic_clean.fillna({'quote_url':0}, inplace=True)

#Changed data type from float64 to int64
BH_tweets_during_pandemic_clean['quote_url'] = BH_tweets_during_pandemic_clean['quote_url'].astype(np.int64)

In [15]:
#Replayce NaN to [] in thumbnail column
BH_tweets_during_pandemic_clean.fillna({'thumbnail':'[]'}, inplace=True)

In [16]:
#There is no NaN except column place which will be removed due to low importance
BH_tweets_during_pandemic_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36155 entries, 0 to 36154
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               36155 non-null  int64         
 1   conversation_id  36155 non-null  int64         
 2   created_at       36155 non-null  object        
 3   date             36155 non-null  datetime64[ns]
 4   time             36155 non-null  object        
 5   timezone         36155 non-null  int64         
 6   user_id          36155 non-null  int64         
 7   username         36155 non-null  object        
 8   name             36153 non-null  object        
 9   place            253 non-null    object        
 10  tweet            36155 non-null  object        
 11  language         36155 non-null  object        
 12  mentions         36155 non-null  object        
 13  urls             36155 non-null  object        
 14  photos           36155 non-null  objec

In [17]:
#Check duplicates
BH_tweets_during_pandemic_clean[BH_tweets_during_pandemic_clean.duplicated(subset = 'id')]

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to


In [18]:
#Column retweet consist of only False values
BH_tweets_during_pandemic_clean['retweet'].value_counts()

False    36155
Name: retweet, dtype: int64

In [19]:
#Removed irrelevant columns
BH_tweets_during_pandemic_clean = BH_tweets_during_pandemic_clean.drop(['conversation_id', 
                                                                        'created_at', 'time', 'timezone', 
                                                                        'language', 'retweet', 'link','place'], 
                                                                       axis = 1)

In [20]:
#Counted number of elements stored as lists in columns
columns_lists = ['mentions', 'urls', 'photos', 'hashtags', 'cashtags', 'thumbnail']
for column in columns_lists:
    empty_list = BH_tweets_during_pandemic_clean[column] == '[]'
    BH_tweets_during_pandemic_clean[column] = BH_tweets_during_pandemic_clean[column].str[1:-1].str.split(',')
    BH_tweets_during_pandemic_clean[column] = BH_tweets_during_pandemic_clean[column].str.len()
    BH_tweets_during_pandemic_clean.loc[empty_list, column] = 0

In [21]:
#Counted number of elements in dictionaries stored in reply_to column
BH_tweets_during_pandemic_clean['reply_to'] = BH_tweets_during_pandemic_clean['reply_to'].str.count('{')

#Changed data type from float64 to int64
BH_tweets_during_pandemic_clean['reply_to'] = BH_tweets_during_pandemic_clean['reply_to'].astype(np.int64)

In [22]:
#There are 2 name missing
BH_tweets_during_pandemic_clean[BH_tweets_during_pandemic_clean['name'].isnull()]

,id,date,user_id,username,name,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,quote_url,video,thumbnail,reply_to
7522,1256981981075632128,2020-05-03,2259884294,stocks_bot,NaN,"How Warren Buffett's 'mistake,' Berkshire Hath...",0,1,0,0,0,0,0,0,0,0,0,0
13996,1266058798394793985,2020-05-28,2259884294,stocks_bot,NaN,Why hedge-fund manager Bill Ackman says he dum...,0,1,0,0,0,0,0,0,0,0,0,0


In [23]:
#Replayce missing names by stocks_bot
BH_tweets_during_pandemic_clean.fillna({'name':'stocks_bot'}, inplace=True)

In [24]:
#Removed hastags, mentions, url and special signs from tweets
for i in range(BH_tweets_during_pandemic_clean.shape[0]):
    BH_tweets_during_pandemic_clean.loc[i, 'tweet'] = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",
                            BH_tweets_during_pandemic_clean.loc[i, 'tweet']).split())

In [25]:
#Last day 2020-10-01 has only 25 tweets, it will be removed
BH_tweets_during_pandemic_clean['date'].value_counts()

2020-05-03    2241
2020-05-02    1684
2020-05-04     898
2020-08-31     862
2020-07-06     575
              ... 
2020-06-14      80
2020-06-27      77
2020-09-27      76
2020-09-20      70
2020-10-01      22
Name: date, Length: 184, dtype: int64

In [26]:
BH_tweets_during_pandemic_clean = BH_tweets_during_pandemic_clean[BH_tweets_during_pandemic_clean['date'] != '2020-10-01']
BH_tweets_during_pandemic_clean['date'].value_counts()

2020-05-03    2241
2020-05-02    1684
2020-05-04     898
2020-08-31     862
2020-07-06     575
              ... 
2020-06-07      81
2020-06-14      80
2020-06-27      77
2020-09-27      76
2020-09-20      70
Name: date, Length: 183, dtype: int64

In [27]:
#Checked if there are tweets that contains only white spaces, numbers or special signs
BH_tweets_during_pandemic_clean[~BH_tweets_during_pandemic_clean["tweet"].str.contains('[A-Za-z]')]

,id,date,user_id,username,name,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,quote_url,video,thumbnail,reply_to


In [28]:
BH_tweets_during_pandemic_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36133 entries, 0 to 36132
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              36133 non-null  int64         
 1   date            36133 non-null  datetime64[ns]
 2   user_id         36133 non-null  int64         
 3   username        36133 non-null  object        
 4   name            36133 non-null  object        
 5   tweet           36133 non-null  object        
 6   mentions        36133 non-null  int64         
 7   urls            36133 non-null  int64         
 8   photos          36133 non-null  int64         
 9   replies_count   36133 non-null  int64         
 10  retweets_count  36133 non-null  int64         
 11  likes_count     36133 non-null  int64         
 12  hashtags        36133 non-null  int64         
 13  cashtags        36133 non-null  int64         
 14  quote_url       36133 non-null  int64         
 15  vi

In [29]:
#Save cleaned data in csv format
BH_tweets_during_pandemic_clean.to_csv('dane/BH_tweets_during_pandemic_clean.csv', index=False)

In [30]:
#Create dataframe for textanalysis for each day
BH_tweets_during_pandemic_textanalysis = pd.date_range(BH_tweets_during_pandemic_clean['date'].min(),
                                                       BH_tweets_during_pandemic_clean['date'].max(),freq='d')
BH_tweets_during_pandemic_textanalysis = pd.DataFrame(BH_tweets_during_pandemic_textanalysis, columns = ['date'])
column_names = ['Short negative factual tweets', 'Short negative personal tweets', 'Short positive factual tweets',
                'Short positive personal tweets', 'Short neutral factual tweets', 'Short neutral personal tweets',
                'Long negative factual tweets', 'Long negative personal tweets', 'Long positive factual tweets',
                'Long positive personal tweets', 'Long neutral factual tweets', 'Long neutral personal tweets',
                'Total amount of tweets']
for column in column_names:
    BH_tweets_during_pandemic_textanalysis[column] = 0
BH_tweets_during_pandemic_textanalysis

,date,Short negative factual tweets,Short negative personal tweets,Short positive factual tweets,Short positive personal tweets,Short neutral factual tweets,Short neutral personal tweets,Long negative factual tweets,Long negative personal tweets,Long positive factual tweets,Long positive personal tweets,Long neutral factual tweets,Long neutral personal tweets,Total amount of tweets
0,2020-04-01,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2020-04-02,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2020-04-03,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2020-04-04,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2020-04-05,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,2020-09-26,0,0,0,0,0,0,0,0,0,0,0,0,0
179,2020-09-27,0,0,0,0,0,0,0,0,0,0,0,0,0
180,2020-09-28,0,0,0,0,0,0,0,0,0,0,0,0,0
181,2020-09-29,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
#Analysis of opinion and number of words using the textblob package
for i in range(BH_tweets_during_pandemic_clean.shape[0]):
    blob = TextBlob(BH_tweets_during_pandemic_clean.loc[i, 'tweet'], analyzer=PatternAnalyzer())
    if len(blob.words) < 30:
        if blob.sentiment[0] < 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short negative factual tweets'] += 1
            else:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short negative personal tweets'] += 1
        elif blob.sentiment[0] > 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short positive factual tweets'] += 1
            else:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short positive personal tweets'] += 1
        else:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short neutral factual tweets'] += 1
            else:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short neutral personal tweets'] += 1
    else:
        if blob.sentiment[0] < 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long negative factual tweets'] += 1
            else:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long negative personal tweets'] += 1
        elif blob.sentiment[0] > 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long positive factual tweets'] += 1
            else:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long positive personal tweets'] += 1
        else:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long neutral factual tweets'] += 1
            else:
                BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long neutral personal tweets'] += 1
    BH_tweets_during_pandemic_textanalysis.loc[BH_tweets_during_pandemic_textanalysis[BH_tweets_during_pandemic_textanalysis['date'] == BH_tweets_during_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Total amount of tweets'] += 1

In [32]:
BH_tweets_during_pandemic_textanalysis

,date,Short negative factual tweets,Short negative personal tweets,Short positive factual tweets,Short positive personal tweets,Short neutral factual tweets,Short neutral personal tweets,Long negative factual tweets,Long negative personal tweets,Long positive factual tweets,Long positive personal tweets,Long neutral factual tweets,Long neutral personal tweets,Total amount of tweets
0,2020-04-01,3,3,17,14,37,2,5,1,13,15,3,1,114
1,2020-04-02,4,5,58,19,28,0,2,3,20,12,3,0,154
2,2020-04-03,2,4,18,17,49,0,4,2,20,21,7,1,145
3,2020-04-04,15,19,60,68,168,3,8,2,23,20,9,0,395
4,2020-04-05,6,2,24,19,53,2,3,0,11,16,3,0,139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,2020-09-26,7,1,14,16,34,1,1,1,16,6,3,0,100
179,2020-09-27,2,2,8,11,24,1,4,1,12,8,3,0,76
180,2020-09-28,8,2,5,19,60,0,1,3,18,14,7,0,137
181,2020-09-29,20,8,19,27,47,0,3,4,30,19,6,0,183


In [33]:
BH_tweets_during_pandemic_textanalysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            183 non-null    datetime64[ns]
 1   Short negative factual tweets   183 non-null    int64         
 2   Short negative personal tweets  183 non-null    int64         
 3   Short positive factual tweets   183 non-null    int64         
 4   Short positive personal tweets  183 non-null    int64         
 5   Short neutral factual tweets    183 non-null    int64         
 6   Short neutral personal tweets   183 non-null    int64         
 7   Long negative factual tweets    183 non-null    int64         
 8   Long negative personal tweets   183 non-null    int64         
 9   Long positive factual tweets    183 non-null    int64         
 10  Long positive personal tweets   183 non-null    int64         
 11  Long n

In [34]:
#Save text analysis data in csv format
BH_tweets_during_pandemic_textanalysis.to_csv('dane/BH_tweets_during_pandemic_textanalysis.csv', index=False)